In [1]:
from __future__ import annotations

from typing import TYPE_CHECKING

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [2]:
from typing import (
    Any,
    AsyncIterator,
    Dict,
    Iterable,
    Iterator,
    List,
    Optional,
    Sequence,
    cast,
)
import rich
from sandbox_agent import debugger
from langchain_core.runnables.schema import StreamEvent


class AnyStr(str):
    def __eq__(self, other: Any) -> bool:
        return isinstance(other, str)


def _with_nulled_run_id(events: Sequence[StreamEvent]) -> List[StreamEvent]:
    """Removes the run ids from events."""
    for event in events:
        assert "run_id" in event, f"Event {event} does not have a run_id."
        assert "parent_ids" in event, f"Event {event} does not have parent_ids."
        assert isinstance(
            event["run_id"], str
        ), f"Event {event} run_id is not a string."
        assert isinstance(
            event["parent_ids"], list
        ), f"Event {event} parent_ids is not a list."

    return cast(
        List[StreamEvent],
        [{**event, "run_id": "", "parent_ids": []} for event in events],
    )


async def _as_async_iterator(iterable: List) -> AsyncIterator:
    """Converts an iterable into an async iterator."""
    for item in iterable:
        yield item


async def _collect_events(
    events: AsyncIterator[StreamEvent], with_nulled_ids: bool = True
) -> List[StreamEvent]:
    """Collect the events and remove the run ids."""
    materialized_events = [event async for event in events]

    if with_nulled_ids:
        events_ = _with_nulled_run_id(materialized_events)
    else:
        events_ = materialized_events
    for event in events_:
        event["tags"] = sorted(event["tags"])
    return events_

In [3]:
from sandbox_agent.ai.workflows import WorkflowFactory

workflow = WorkflowFactory.create()
# workflow.pretty_print()


from IPython.display import Image, display

display(Image(workflow.get_graph(xray=True).draw_mermaid_png()))

ModuleNotFoundError: No module named 'src'